In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('feature_engineered_dataset.csv')

In [6]:
df.head(2)

,DATE,WEEK,MONTH,GYM TYPE,BODYWEIGHT,PHASE,EXERCISE,MUSCLE GROUP,WEIGHT,REPS,VOLUME,ESTIMATED 1RM,RELATIVE STRENGTH
0,2025-04-07,15,Apr,College Gym,70.85,Bulk,Squats,Legs,27.5,12,330.0,38.5,0.543402
1,2025-04-07,15,Apr,College Gym,70.85,Bulk,Squats,Legs,30.0,9,270.0,39.0,0.550459


In [21]:
df.shape

(1804, 13)

In [19]:
weekly_df = (
    df
    .groupby(
        ["WEEK", "MONTH", "MUSCLE GROUP", "GYM TYPE", "PHASE"],
        as_index=False
    )
    .agg(
        AVG_E1RM=("ESTIMATED 1RM", "mean"),
        AVG_RELATIVE_STRENGTH=("RELATIVE STRENGTH", "mean"),
        TOTAL_VOLUME=("VOLUME", "sum")
    )
)


In [24]:
weekly_df = weekly_df.sort_values(
    ["MUSCLE GROUP", "GYM TYPE", "PHASE", "WEEK"]
)


In [30]:
weekly_df["E1RM_CHANGE"] = (
    weekly_df
    .groupby(["MUSCLE GROUP", "GYM TYPE", "PHASE"])["AVG_E1RM"]
    .diff()
)


In [32]:
weekly_df["STRENGTH_TREND"] = np.where(
    weekly_df["E1RM_CHANGE"] > 0.5,
    "INCREASING",
    np.where(
        weekly_df["E1RM_CHANGE"] < -0.5,
        "DECREASING",
        "STABLE"
    )
)


In [33]:
weekly_df.head(5)

,WEEK,MONTH,MUSCLE GROUP,GYM TYPE,PHASE,AVG_E1RM,AVG_RELATIVE_STRENGTH,TOTAL_VOLUME,E1RM_CHANGE,STRENGTH_TREND
0,15,Apr,Back,College Gym,Bulk,39.641667,0.562293,2967.5,NaN,STABLE
11,17,Apr,Back,College Gym,Bulk,41.628788,0.592728,3162.5,1.987121,INCREASING
19,19,May,Back,College Gym,Bulk,32.944444,0.457880,2145.0,-8.684343,DECREASING
23,20,May,Back,College Gym,Bulk,41.666667,0.576302,2100.0,8.722222,INCREASING
29,21,May,Back,College Gym,Bulk,46.138889,0.632040,2707.5,4.472222,INCREASING


BULK RULE

In [53]:
bulk_issue = (
    (weekly_df["PHASE"] == "BULK") &
    (weekly_df["STRENGTH_TREND"] != "INCREASING")
)

CUT RULE

In [ ]:
cut_issue = (
    (weekly_df["PHASE"] == "CUT") &
    (weekly_df["REL_STRENGTH_CHANGE"] < 0)
)

In [54]:
weekly_df["RECOVERY_FLAG"] = np.where(
    bulk_issue | cut_issue,
    "UNDER_RECOVERED",
    "RECOVERED"
)

In [55]:
weekly_df = weekly_df.rename(columns={'MUSCLE GROUP': 'MUSCLE_GROUP', 'GYM TYPE': 'GYM_TYPE'})

In [59]:
weekly_df['RECOVERY_FLAG'].value_counts()

RECOVERY_FLAG
RECOVERED    214
Name: count, dtype: int64

In [61]:
weekly_df

,WEEK,MONTH,MUSCLE_GROUP,GYM_TYPE,PHASE,AVG_E1RM,AVG_RELATIVE_STRENGTH,TOTAL_VOLUME,E1RM_CHANGE,STRENGTH_TREND,REL_STRENGTH_CHANGE,RECOVERY_FLAG
0,15,Apr,Back,College Gym,Bulk,39.641667,0.562293,2967.5,NaN,STABLE,NaN,RECOVERED
11,17,Apr,Back,College Gym,Bulk,41.628788,0.592728,3162.5,1.987121,INCREASING,0.030434,RECOVERED
19,19,May,Back,College Gym,Bulk,32.944444,0.457880,2145.0,-8.684343,DECREASING,-0.134848,RECOVERED
23,20,May,Back,College Gym,Bulk,41.666667,0.576302,2100.0,8.722222,INCREASING,0.118423,RECOVERED
29,21,May,Back,College Gym,Bulk,46.138889,0.632040,2707.5,4.472222,INCREASING,0.055737,RECOVERED
...,...,...,...,...,...,...,...,...,...,...,...,...
155,40,Sep,Triceps,College Gym,Cut,45.638889,0.692548,1915.0,1.527778,INCREASING,0.026720,RECOVERED
161,42,Oct,Triceps,College Gym,Cut,33.812500,NaN,1965.0,-11.826389,DECREASING,NaN,RECOVERED
168,44,Oct,Triceps,College Gym,Cut,27.638889,0.422290,1000.0,-6.173611,DECREASING,NaN,RECOVERED
176,45,Nov,Triceps,College Gym,Cut,49.437500,0.761684,3315.0,21.798611,INCREASING,0.339394,RECOVERED


In [60]:
weekly_df.to_csv("workout_weekly_analysis.csv", index=False)
